In [1]:
#Imports
import requests
import pandas as pd
from pandas import json_normalize
import mysql.connector
import pymysql
from sqlalchemy import create_engine

## COVID-19 Daily Summary Statistics by Country

In [2]:
#Fetching the daily covid data from Postman API
api = r'https://api.covid19api.com/summary'
r = requests.get(api)

In [3]:
#Flattening JSON to Dataframe
daily_df = json_normalize(r.json(), record_path = 'Countries')

## Contry Data

In [4]:
daily_df['country_id'] = daily_df.groupby(['Country']).ngroup()

In [5]:
country_data = daily_df[['Country', 'country_id']]

In [6]:
#regions_url = 'https://raw.githubusercontent.com/dbouquin/IS_608/master/NanosatDB_munging/Countries-Continents.csv'
#region_data = pd.read_csv(regions_url)

In [7]:
#country_data = pd.merge(country_data, region_data, how = 'left', on='Country')

## COVID-19 Vaccinations By Country

In [8]:
vaccines_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'
vaccinations = pd.read_csv(vaccines_url)

In [13]:
vaccinations.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,Country,country_id,_merge
0,Albania,ALB,2021-01-10,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Albania,1.0,both
1,Albania,ALB,2021-01-11,NaN,NaN,NaN,NaN,64.0,NaN,NaN,NaN,22.0,Albania,1.0,both
2,Albania,ALB,2021-01-12,128.0,128.0,NaN,NaN,64.0,0.00,0.00,NaN,22.0,Albania,1.0,both
3,Albania,ALB,2021-01-13,188.0,188.0,NaN,60.0,63.0,0.01,0.01,NaN,22.0,Albania,1.0,both
4,Albania,ALB,2021-01-14,266.0,266.0,NaN,78.0,66.0,0.01,0.01,NaN,23.0,Albania,1.0,both


In [9]:
vaccinations = pd.merge(vaccinations, country_data[['Country', 'country_id']], how = 'left', left_on = 'location', right_on = 'Country', indicator=True)

In [11]:
vaccinations[vaccinations['_merge'] == 'left_only'].value_counts('location')

location
World                       93
Wales                       92
England                     92
Scotland                    92
Northern Ireland            92
Russia                      90
United States               86
Czechia                     79
European Union              79
Cayman Islands              77
Gibraltar                   65
Bermuda                     60
Isle of Man                 54
Guernsey                    48
Jersey                      46
Faeroe Islands              43
Macao                       34
Greenland                   34
Turks and Caicos Islands    30
Anguilla                    23
Hong Kong                   23
South Korea                 19
Montserrat                  16
Venezuela                   16
Falkland Islands            16
Cote d'Ivoire               15
Iran                        10
Northern Cyprus              9
Vietnam                      9
Saint Helena                 1
dtype: int64

## Data Cleaning/Wrangling

In [10]:
daily_df = daily_df.drop(['ID', 'Country', 'Slug', 'NewRecovered', 'TotalRecovered', 'CountryCode'], axis = 1)
daily_df.columns = ['new_confirmed', 'total_confirmed', 'new_deaths', 'total_deaths', 'date', 'country_id']
daily_df['date'] = daily_df['date'].apply(lambda x: x[0:10])

In [11]:
country_data.columns = ['country', 'country_id', 'region']

In [12]:
vaccinations = vaccinations.drop(['iso_code', 'daily_vaccinations_raw', 'location', 'Country'], axis = 1)

## Storing the data in a MySQL database

In [13]:
hostname = "localhost"
uname = "root"
pwd = "Vadhavula@11"
dbname = "covid_data"

In [14]:
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                .format(host=hostname, db=dbname, user=uname, pw=pwd))

In [15]:
connection = engine.connect()

In [16]:
connection.execute("CREATE DATABASE IF NOT EXISTS covid_data")

In [17]:
daily_df.head()

,new_confirmed,total_confirmed,new_deaths,total_deaths,date,country_id
0,10,55995,1,2460,2021-03-17,0
1,475,118492,17,2077,2021-03-17,1
2,130,115540,5,3045,2021-03-17,2
3,30,11319,0,113,2021-03-17,3
4,39,21446,1,522,2021-03-17,4


In [18]:
connection.execute('''
CREATE TABLE IF NOT EXISTS daily_data (
    country_id INTEGER NOT NULL,
    date DATETIME NOT NULL,
    new_confirmed INTEGER,
    total_confirmed INTEGER NOT NULL,
    new_deaths INTEGER,
    total_deaths INTEGER NOT NULL
)
''')

In [19]:
connection.execute('''
CREATE TABLE IF NOT EXISTS countries (
    country_id INTEGER NOT NULL,
    country VARCHAR(50) NOT NULL,
    region VARCHAR(50),
    PRIMARY KEY (country_id)
)
''')

In [20]:
connection.execute('''
CREATE TABLE IF NOT EXISTS vaccinations (
    country_id INTEGER,
    date DATETIME NOT NULL,
    total_vaccinations INTEGER,
    people_vaccinated INTEGER,
    people_fully_vaccinated INTEGER,
    daily_vaccinations INTEGER,
    total_vaccinations_per_hundred DECIMAL(5,2),
    people_vaccinated_per_hundred DECIMAL(5,2),
    people_fully_vaccinated_per_hundred DECIMAL(5,2),
    daily_vaccinations_per_million DECIMAL(9,2)
)
''')

In [21]:
daily_df.to_sql(con = engine, name = 'daily_data', index = False, if_exists='append')

In [22]:
connection.execute('ALTER TABLE daily_data ADD id SERIAL PRIMARY KEY')
connection.execute('ALTER TABLE daily_data CHANGE id id SERIAL NOT NULL FIRST')

In [23]:
connection.execute('ALTER TABLE vaccinations ADD id SERIAL PRIMARY KEY')
connection.execute('ALTER TABLE vaccinations CHANGE id id SERIAL NOT NULL FIRST')

In [24]:
country_data.to_sql(con = engine, name = 'countries', index = False, if_exists = 'replace')

In [27]:
vaccinations.to_sql(con = engine, name = 'vaccinations', index = False, if_exists = 'append')

In [28]:
connection.close()